import the pre rquisites 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

set up the web driver

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

open the web page using selenium

In [ ]:
driver.get("https://www.healthyfood.com/healthy-recipes/")


In [ ]:
scroll_times = 100

scroll the webpage to load the data

In [ ]:
for _ in range(scroll_times):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7) 

To get the source code after scrolling

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')


In [ ]:
links = [
    h3.a['href']
    for h3 in soup.find_all('h3', class_='entry-title')
    if h3.a and h3.a.has_attr('href')
]

In [ ]:
links

In [ ]:
len(links)

In [ ]:
driver.quit()

In [ ]:
import json
with open("links.json","w") as f:
    json.dump(links,f)

In [ ]:
def get_name(link):
    name_recipe=[]
    webpage=requests.get(link).text
    soup=BeautifulSoup(webpage,'lxml')
    name_recipe.append(soup.find_all("h1",class_="title fn")[0].text)
    return name_recipe


In [ ]:
# Find the div with class 'ingredient_list'
def get_ingredients(link):
    webpage=requests.get(link).text
    soup=BeautifulSoup(webpage,'lxml')
    ingredient_section = soup.find('div', id='ingredients-list')
    ingredient = [li.get_text(strip=True) for li in ingredient_section.find_all('li', class_='ingredient')]
    return ingredient


In [ ]:
def get_nutrients(link):
        webpage=requests.get(link).text
        soup=BeautifulSoup(webpage,'lxml')
        nutritients={}
        nutritional_section=soup.find('div',class_="nutritional")
        for li in nutritional_section.find_all('li'):
                # Extract nutrient name and value (grams)
                nutrient_name = li.find('span', class_='big-nut').text
                nutrient_value = li.get_text(strip=True).replace(nutrient_name, '').strip()  # Remove the nutrient name to get the value
        
                # Add the nutrient and its value to the dictionary
                nutritients[nutrient_name] = nutrient_value
        return nutritients

In [ ]:
def get_instruction(link):
    webpage=requests.get(link).text
    soup=BeautifulSoup(webpage,'lxml')
    instructions_section = soup.find('div', id='fld_instructions_and_steps')
    instruction = []
    for step in instructions_section.find_all('p'):
        instruction.append(step.get_text(strip=True))
    return instruction

In [ ]:
def get_time(link):
    webpage=requests.get(link).text
    soup=BeautifulSoup(webpage,'lxml')
    time_tag = soup.find('div', class_='cooking_time_text')
    if time_tag:
        time_to_make = time_tag.get_text(strip=True).replace("Time to make:", "").strip()
    return time_to_make


In [ ]:
def servesize(link):
    webpage = requests.get(link).text
    soup = BeautifulSoup(webpage, 'lxml')
    serves = soup.find('span', class_='serves')
    if serves:
        serve_size = serves.get_text(strip=True).replace("Serves:", "").strip()
    else:
        serve_size="null"
    return serve_size


In [ ]:
def gettags(link):
    webpage=requests.get(link).text
    soup=BeautifulSoup(webpage,'lxml')
    health_tags = []
    health_info_div = soup.find('div', class_='inner health-info')
    if health_info_div:
        for a in health_info_div.find_all('a', class_='recipe_circle'):
            title = a.get('title')
            if title and "Click for more" in title:
                tag = title.replace("Click for more", "").replace("recipes", "").strip()
                health_tags.append(tag)
    return health_tags 



In [ ]:
len(links)

In [ ]:
name=[]
tags=[]
ingredients=[]
serving_size=[]
make_time=[]
nutrition=[]
instructions=[]  

In [ ]:
for link in links:
    name.append(get_name(link))
    tags.append(gettags(link))
    ingredients.append(get_ingredients(link))
    serving_size.append(servesize(link))
    make_time.append(get_time(link))
    nutrition.append(get_nutrients(link))
    instructions.append(get_instruction(link))

In [ ]:
recipes={
    "name":name,
    "tags":tags,
    "ingredients":ingredients,
    "serving_size":serving_size,
    "make_time":make_time,
    "nutrition":nutrition,
    "instructions":instructions
}

In [ ]:
df=pd.DataFrame(recipes)

In [ ]:
df.head()

In [ ]:
df.to_csv('recipes.csv',index=False)

In [ ]:
df.info()